In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
%cd '/content/drive/My Drive'

Mounted at /content/drive
/content/drive/My Drive


In [2]:
os.chdir('./COMP7607NLP_Project/subnetwork-probing/code')
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.6 MB/s 
     |████████████████████████████████| 182 kB 72.6 MB/s 
     |████████████████████████████████| 7.6 MB 47.9 MB/s 


In [3]:
import sys
from visualize import visualize_head_sparsity, visualize_dense_sparsity, visualize_layer_attn_sparsity
from matplotlib import pyplot as plt
plt.switch_backend('agg')
import numpy as np
from classifiers import BertEncoder
from bert import WordLevelBert
from masked_bert import MaskedWordLevelBert
from train import train_ner, train_ud, train_pos, save_mask_scores
from visualize import visualize_head_sparsity, visualize_dense_sparsity, visualize_layer_attn_sparsity

Using CUDA!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
mask_lr_base = 0.2
lr_base = 5e-05
lambda_init = 0
lambda_final = 125
verbose = True
!nvidia-smi

Sat Nov 19 07:08:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
#choose the UD task, pretrained setting,prune method
task = 'UD'
setting = 'pretrained'
method = 'prune'
params_list = ((768,768), (768,192), (768,24), (768,6), 
                               (768,1), (192,1), (24,1), (6,1), (1,1))
params = params_list[4]
#rank = params
epochs = 20
out_w_per_mask, in_w_per_mask = params
model_name = 'bert-base-multilingual-uncased'
masked = True
kwargs = {'lambda_init' : lambda_init, 'lambda_final' : lambda_final, 
                          'epochs' : epochs, 'lr_base' : lr_base, 'mask_lr_base' : mask_lr_base, 
                          'verbose' : verbose, 'masked' : masked}


In [6]:
#subnetwork 
bert = MaskedWordLevelBert(model_name, out_w_per_mask, in_w_per_mask)
bert.freeze_bert()
bert_encoder = BertEncoder(bert, mlp1 = False)
#bert.reset_weights(encoder_only = False)

#mlp1
#bert = WordLevelBert(model_name)
#bert.freeze_bert()
#bert_encoder = BertEncoder(bert, mlp1 = True, rank = rank)
#bert.reset_weights(encoder_only = False)
#fine tuning
#bert = WordLevelBert(model_name)
#bert_encoder = BertEncoder(bert, mlp1 = False)

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
log, model = train_ud(bert_encoder, '../../UD_English/en-ud-train.conllu', 
                                          '../../UD_English/en-ud-dev.conllu', **kwargs)

Removed length 159
Removed length 135
lambda_init: 0, lambda_final: 125, lambda_startup_frac: 0.25, lambda_warmup_frac: 0.5
lr_base: 5e-05, mask_lr_base: 0.2, lr_warmup_frac: 0.1, epochs: 20, batch_size: 32


/content/drive/MyDrive/COMP7607NLP_Project/subnetwork-probing/code/bert.py:103: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  features_packed = features.masked_select(all_end_mask)
/content/drive/MyDrive/COMP7607NLP_Project/subnetwork-probing/code/classifiers.py:140: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/cuda/Indexing.cu:1239.)
  dep_W_headT.data.masked_fill_(pad_mask.byte(), -float('inf'))
/content/drive/MyDrive/COMP7607NLP_Project/subnetwork-probing/code/datasets.py:110: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:27.)
  gold = gold.masked_select(mask.byte())
/content/drive/MyD

Log: {'dev_acc': 0.004576569395988591, 'loss_val': 14.351385116577148, 'reg_val': 0.9598479866981506, 'pct_binary': 0.0}
Log: {'dev_acc': 0.04968556547072748, 'loss_val': 8.52513313293457, 'reg_val': 0.9578993320465088, 'pct_binary': 0.0}
Log: {'dev_acc': 0.10540048347523208, 'loss_val': 6.450857162475586, 'reg_val': 0.9562564492225647, 'pct_binary': 0.0}
Log: {'dev_acc': 0.14416885441321772, 'loss_val': 6.625941276550293, 'reg_val': 0.9546688199043274, 'pct_binary': 0.0}
Log: {'dev_acc': 0.15602645724238276, 'loss_val': 6.179214000701904, 'reg_val': 0.9529857635498047, 'pct_binary': 0.0}
Log: {'dev_acc': 0.20598603398109094, 'loss_val': 5.524906158447266, 'reg_val': 0.9504632949829102, 'pct_binary': 9.04224537037037e-06}
Log: {'dev_acc': 0.22855633162201142, 'loss_val': 4.857707977294922, 'reg_val': 0.9467827677726746, 'pct_binary': 2.712673611111111e-05}
Log: {'dev_acc': 0.2463104770823853, 'loss_val': 4.371265411376953, 'reg_val': 0.9421998262405396, 'pct_binary': 9.946469907407407e

In [ ]:
path = '../../Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
save_mask_scores(model, log, base = path)

Saving to ../../Results/bert-base-multilingual-uncased_NER_setting=resetall_method=mlp1_params=768_dev_acc=55.8578.pt...
Done saving


In [ ]:
for key in log[0].keys():
    plt.plot(np.arange(len(log)), [a[key] for a in log], color='blue')
    plt.title(key)
    plt.savefig('{}_{}_graph.png'.format(path, key))
    plt.show(block=False)
    plt.close()


In [ ]:
sparsities = np.array([bert.get_sparsity_layer_attn(l).item() for l in range(12)])
sparsities = sparsities / np.sum(sparsities)
sparsities

array([0.27118644, 0.05932203, 0.30084745, 0.05084746, 0.12288136,
       0.11864407, 0.06779661, 0.        , 0.00423729, 0.00423729,
       0.        , 0.        ])

In [ ]:
path = '../../Results/{}_{}_setting={}_method={}_params={}'.format(
                   model_name, task, setting, method, str(params).replace(' ',''))
if masked:
     visualize_head_sparsity(bert, path, block = False)
     visualize_dense_sparsity(bert, path, block = False)
     visualize_layer_attn_sparsity(bert, path, block = False)
     print("Percentage of elements within 0.01 of 0 or 1: {:.5f}".format(bert.compute_binary_pct()))

Percentage of elements within 0.01 of 0 or 1: 0.99958
